# Basic tools use

Various LLM use cases can utilize function/tool calls to extend LLM capabilities. Draive comes with a dedicated solution to prepare tools and control its execution.

## Tool definition

Let's start by defining a simple tool. Tool is usually a function which is explained to LLM and can be requested to be used. Tools can require generation of arguments and have to return some value. Tools defined within draive are python async function annotated with `tool` wrapper.

In [1]:
from draive import tool


@tool # simply annotate function as a tool, tools can have arguments using basic types
async def current_time(location: str) -> str:
    # return result of tool execution, we are using fake time here
    return f"Time in {location} is 9:53:22"

## Tool use

After defining a tool we can use it in multiple scenarios to extend LLM capabilities. All of the tool arguments will be validated when calling. It includes type check and any additional validation if defined. You can still use it as a regular function, although, all tools have to be executed within draive context scope. 

In [2]:
from draive import ctx

async with ctx.new():
    # we can still use it as a regular function
    # but it has to be executed within context scope
    print(await current_time(location="London"))

# await current_time(location="London") # error! out of context

Time in London is 9:53:22


The biggest benefit of defining a tool is when using LLM. We can tell the model to use available tools to extend its capabilities. Higher level interfaces automatically handle tool calls and going back with its result to LLM to receive the final result. We can see how it works within a simple text generator. Tools are provided within a `Toolbox` object which allows customizing tools execution. You can prepare a `Toolbox` each time it is used or reuse preconstructed one. We will use OpenAI GPT model as it natively supports tool use. Make sure to provide .env file with `OPENAI_API_KEY` key before running. 

In [3]:
from draive import LMM, OpenAIClient, Toolbox, generate_text, load_env, openai_lmm_invocation

load_env()

async with ctx.new(
    # define used LMM to be OpenAI within the context
    state=[LMM(invocation=openai_lmm_invocation)],
    dependencies=[OpenAIClient],
):
    result: str = await generate_text(
        instruction="You are a helpful assistant",
        input="What is the time in New York?",
        tools=Toolbox(current_time),
    )

    print(result)

The current time in New York is 9:53:22.


## Tool details

Tools can be customized and extended in various ways depending on use case. First of all we can customize tool arguments and help LLM to better understand how to use given tool.

In [4]:
from draive import Argument


@tool( # this time we will use additional arguments within tool annotation
    # we can define an alias used as the tool name when explaining it to LLM,
    # default name is the name of python function
    name="fun_fact",
    # additionally we can explain the tool purpose by using description
    description="Find a fun fact in a given topic",
)
async def customized(
    # we can also annotate arguments to provide even more details
    # and specify argument handling logic
    arg: str = Argument(
        # we can alias each argument name
        alias="topic",
        # further describe its use
        description="Topic of a fact to find",
        # provide default value or default value factory
        default="random",
        # end more, including custom validators
    ),
) -> str:
    return f"{arg} is very funny on its own!"

# we can examine tool specification which is similar to
# how `State` and `DataModel` specification/schema is built
print(customized.specification)

{'type': 'function', 'function': {'name': 'fun_fact', 'parameters': {'type': 'object', 'properties': {'topic': {'type': 'string', 'description': 'Topic of a fact to find'}}, 'required': []}, 'description': 'Find a fun fact in a given topic'}}


## Toolbox

We have already used a Toolbox to ask for an extended LLM completion. However Toolbox allows us to specify some additional details regarding the tools execution like the tool calls limit or a tool suggestion.

In [5]:
async with ctx.new(
    # define used LMM to be OpenAI within the context
    state=[LMM(invocation=openai_lmm_invocation)],
    dependencies=[OpenAIClient],
):
    result: str = await generate_text(
        instruction="You are a funny assistant",
        input="What is the funny thing about LLMs?",
        tools=Toolbox(
            # we can define any number of tools within a toolbox
            current_time,
            customized,
            # we can limit how many tool calls are allowed
            # before the final result is returned
            recursive_calls_limit=2,
            # we can also force any given tool use within the first LLM call
            suggest=customized,
        ),
    )

    print(result)

LLMs is very funny on its own!


## Metrics

All of the tool usage is automatically traced within scope metrics. We can see the details about their execution when using a logger:

In [6]:
from draive import setup_logging

setup_logging("tools_logs")

async with ctx.new(
    "tools_logs",
    # define used LMM to be OpenAI within the context
    state=[LMM(invocation=openai_lmm_invocation)],
    dependencies=[OpenAIClient],
):
    result: str = await generate_text(
        instruction="You are a funny assistant",
        input="What is the funny thing about LLMs?",
        # we will now be able to see what tools were used
        # and check the details about its execution
        tools=Toolbox(
            current_time,
            customized,
        ),
    )

    print(f"\nResult:\n{result}\n")

07/Jun/2024:14:09:27 +0000 [INFO] [tools_logs] [54aac040908840cca77e6807b27b8a68|tools_logs] started...
07/Jun/2024:14:09:27 +0000 [INFO] [tools_logs] [54aac040908840cca77e6807b27b8a68|lmm_generate_text] started...
07/Jun/2024:14:09:27 +0000 [INFO] [tools_logs] [54aac040908840cca77e6807b27b8a68|openai_lmm_completion] started...
07/Jun/2024:14:09:27 +0000 [DEBUG] [tools_logs] [54aac040908840cca77e6807b27b8a68|openai_lmm_completion] Requested OpenAI lmm
07/Jun/2024:14:09:28 +0000 [INFO] [tools_logs] [54aac040908840cca77e6807b27b8a68|openai_lmm_completion] ...finished after 0.91s
07/Jun/2024:14:09:28 +0000 [DEBUG] [tools_logs] [54aac040908840cca77e6807b27b8a68|lmm_generate_text] Received text generation tool calls
07/Jun/2024:14:09:28 +0000 [INFO] [tools_logs] [54aac040908840cca77e6807b27b8a68|fun_fact] started...
07/Jun/2024:14:09:28 +0000 [INFO] [tools_logs] [54aac040908840cca77e6807b27b8a68|fun_fact] ...finished after 0.00s
07/Jun/2024:14:09:28 +0000 [INFO] [tools_logs] [54aac040908840